In [1]:
import sys
sys.path.append('../')
import os
os.environ["OMP_NUM_THREADS"] = '1'
import argparse

In [2]:
parser = argparse.ArgumentParser("OVG-Nav")


parser.add_argument('--model_name', default='cm_0706/0706_mp3d21_pano_goalscore_adjloss0.5_use_cm_maxdist30.0_lr0.01', type=str)
parser.add_argument("--value_model", type=str, default='../goal_dist_pred/logs/{}/model_10.pth')
parser.add_argument('--vis_feat_dim', default=512, type=int)
parser.add_argument('--max_dist', default=30., type=float)
parser.add_argument('--use_cm_score', default=True, type=bool)
parser.add_argument('--dataset', type=str, default='mp3d')
parser.add_argument('--goal_cat', type=str, default='mp3d_21')
parser.add_argument("--cm_type", type=str, default="comet")

# Misc
parser.add_argument('--seed', type=int, default=1, help="random seed (default: 1)")
parser.add_argument('--gpu', type=str, default='9', help="which gpu devices to use")
parser.add_argument("--model_gpu", type=str, default="0")


parser.add_argument('--data-dir', default='/Dataset/cm_graph/mp3d/0704/21cat_relative_pose_step_by_step_pano_edge1.0_v2', type=str)
parser.add_argument("--floorplan_data_dir", type=str, default='/Dataset/habitat/data/floorplans')
parser.add_argument('--save_dir', default='visualize_graph/{}', type=str)


args = parser.parse_args([])

args.value_model = args.value_model.format(args.model_name)
args.save_dir = args.save_dir.format(args.model_name)


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches


import json
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import pickle

import habitat_sim


from utils.visualizations.maps import get_topdown_map_from_sim, to_grid, TopdownView
from utils.graph_utils.graph_pano_cs import GraphMap
from utils.obj_category_info import obj_names, gibson_goal_obj_names, mp3d_goal_obj_names, room_names, mp3d_room_names, d3_40_colors_rgb
from goal_dist_pred.model_value_graph_0607 import TopoGCN_v2_pano_goalscore as ValueModel
from goal_dist_pred.dataloader_batch_graph_data_0607 import Batch_traj_DataLoader_pano_goalscore as Batch_traj_DataLoader
from navigation.configs.settings_pano_navi import make_settings, make_cfg

ModuleNotFoundError: No module named 'habitat_sim'

## Initialize settings

### Model initialization

In [ ]:
value_model = ValueModel(args)
value_model = nn.DataParallel(value_model).cuda()
checkpoint = torch.load(args.value_model)
value_model.load_state_dict(checkpoint)
value_model.eval()
print("Successfully load the value model")
print("Model: {}".format(args.model_name))

### Data loader


In [ ]:
val_envs = [os.path.join(args.data_dir, 'val', name) for name in os.listdir(os.path.join(args.data_dir, 'val'))]
val_envs.sort()
val_list = []
for env in val_envs:
    val_list = val_list + [os.path.join(env, x) for x in os.listdir(env)]
val_list.sort()
val_batch_num = int(len(val_list))
val_dataset = Batch_traj_DataLoader(args, val_list)

## Graph visualization functions

In [ ]:
def get_vis_grid_pose(pose, vis_map_info):
    pose = np.array(pose) + np.array(vis_map_info['bias_position'])
    realworld_y, realword_x = pose[2], pose[0]
    grid_resolution = vis_map_info['grid_resolution']
    
    lower_bound, upper_bound = vis_map_info['env_bound']
    grid_size = (
        abs(upper_bound[2] - lower_bound[2]) / grid_resolution[0],
        abs(upper_bound[0] - lower_bound[0]) / grid_resolution[1],
    )
    grid_y = int((realworld_y - lower_bound[2]) / grid_size[0])
    grid_x = int((realworld_x - lower_bound[0]) / grid_size[1])
    
    return (grid_x, grid_y)

def vis_pos_on_topdown_map(pos, vis_map, vis_map_info, color=(255, 0, 0)):
    vis_map = vis_map.copy()
    node_grid = get_vis_grid_pose(pos, vis_map_info)
    vis_map = cv2.rectangle(vis_map, (node_grid[0] - 8, node_grid[1] - 8), (node_grid[0] + 8, node_grid[1] + 8),
                            color, -1)
    return vis_map


def vis_topdown_graph_map(vis_map, graph_map, obj_value, vis_map_info, vis_obj_score=None, curr_node_id=None):
    cmap = mcolors.LinearSegmentedColormap.from_list("mycmap", ['darkblue', 'yellow'])
    node_list = list(graph_map.node_by_id.values())

    for edge in list(graph_map.edges):
        pos1 = np.array(edge.nodes[0].pos) if edge.nodes[0].vis_pos is None else np.array(edge.nodes[0].vis_pos)
        pos2 = np.array(edge.nodes[1].pos) if edge.nodes[1].vis_pos is None else np.array(edge.nodes[1].vis_pos)
        node_grid1 = get_vis_grid_pose(pos1, vis_map_info)
        node_grid2 = get_vis_grid_pose(pos2, vis_map_info)
        vis_map = cv2.line(vis_map, node_grid1, node_grid2, (0, 64, 64), 5)
        edge.draw = True


    for idx, node in enumerate(node_list):

        node_pos = np.array(node.pos) if node.vis_pos is None else np.array(node.vis_pos)

        node_grid = get_vis_grid_pose(node_pos, vis_map_info)
        color = (0, 255, 0)
        
        color_value = np.array(cmap(obj_value[idx]))
        color_value = color_value[:3] *255
        color_value = np.around(color_value).astype(np.uint8)
        cand_color = tuple(color_value)
        cand_color = tuple(int(c) for c in cand_color)
        goal_color = (255, 255, 0)
        if node.visited:
            if node.nodeid == curr_node_id:
                vis_map = cv2.circle(vis_map, node_grid, 10, (255,0,0), -1)
            else:
                vis_map = cv2.circle(vis_map, node_grid, 10, color, -1)
        else:
            vis_map = cv2.circle(vis_map, node_grid, 10, color=cand_color, thickness=-1)

        node.draw = True


    return vis_map


def save_graph_on_topdown_map(vis_map, value_type, save_name, goal_name, env_name, save=False):

    txt = '[{}] goal: {}, value type: {}, '.format(env_name, goal_name, value_type)
    if save:
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        color = (255, 255, 255)
        thickness = 2
        text_size = cv2.getTextSize(txt, font, font_scale, thickness)[0]
        text_position = (10, vis_map.shape[0] + text_size[1] * 2 + 10)
        canvas_height = vis_map.shape[0] + text_size[1] * 2 + 40
        canvas_width = vis_map.shape[1]
        canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
        canvas[:vis_map.shape[0], :] = vis_map
        canvas = cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB)
        cv2.putText(canvas, txt, text_position, font, font_scale, color, thickness)
        cv2.imwrite(save_name, canvas)
    else:
        plt.imshow(vis_map)
        plt.figtext(0.5, 0.01, txt, ha='center', fontsize=12, va='bottom')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    return


def save_comparison_on_topdown_map(gt_map, cm_map, value_map, save_name, goal_name, env_name, acc, save=False):
    
    
    
    txt = '[{}]   Goal: {}, value acc: {:.4f}, rank acc: {}, pred diff: {:.4f}'.format(env_name, goal_name, acc['value_acc'], acc['rank_acc'], acc['pred_diff'])
    
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    sub1 = axes[0]
    sub1.imshow(gt_map)
    sub1.set_title('GT')
    sub1.axis('off')

    sub2 = axes[1]
    sub2.imshow(cm_map)
    sub2.set_title('commonsense score')
    sub2.axis('off')

    sub3 = axes[2]
    sub3.imshow(value_map)
    sub3.set_title('object value')
    sub3.axis('off')

    # Add a bounding box around the entire subplot grid
    for ax in [sub1, sub2, sub3]:
        rect = patches.Rectangle((0, 0), 1, 1, edgecolor='black', linewidth=2, fill=False, transform=ax.transAxes)
        ax.add_patch(rect)


    fig.text(0.5, -0.001, txt, ha='center', fontsize=12, va='bottom')
#     plt.tight_layout()
    
    
    if save:
        plt.savefig(save_name, dpi=300)
    else:
        plt.show()
    plt.close()
    return



## Inference

### prediction

In [ ]:
data_name = val_list[10]

def pred_value(data_name):
    with open(f'{data_name}/graph.pkl', 'rb') as f:
        graph_data = pickle.load(f)

    with torch.no_grad():
        data = val_dataset.load_data(data_name)
        features = data['node_features'].cuda()
        info_features = data['node_info_features'].cuda()
        goal_features = data['node_goal_features'].cuda()
        adj_mtx = data['adj_mtx'].cuda()
        node_goal_dists = data['node_goal_dists'].cuda()
        goal_idx = data['goal_idx']

        pred_dist = value_model(features, goal_features, info_features, adj_mtx)
    
    return graph_data, data, features, info_features, goal_features, adj_mtx, node_goal_dists, goal_idx, pred_dist

graph_data, data, features, info_features, goal_features, adj_mtx, node_goal_dists, goal_idx, pred_dist = pred_value(data_name)

### evaluation

In [ ]:
def eval_result(data, pred_dist, node_goal_dists):
    value_acc = torch.mean(torch.where(abs(pred_dist - node_goal_dists) <= 0.1, 1, 0).float())
    if node_goal_dists.size()[0] >= 3:
        topk_list = torch.topk(node_goal_dists, 3, dim=0).indices
    else:
        topk_list = torch.topk(node_goal_dists, node_goal_dists.size()[0], dim=0).indices
    rank_acc = float(torch.argmax(pred_dist, dim=0) in topk_list)
    pred_diff = np.linalg.norm(data['node_pose'][torch.argmax(pred_dist)] - data['node_pose'][torch.argmax(node_goal_dists)])

    acc = {
        'value_acc': value_acc,
        'rank_acc': rank_acc,
        'pred_diff': pred_diff
    }
    return acc

acc = eval_result(data, pred_dist, node_goal_dists)

## Visualization

### Visualize value graph

In [4]:
def load_floor_map(graph_data, data_dir):
    env_name = graph_data.env_data['env_name']
    level = graph_data.env_data['level']
    bias_position = graph_data.env_data['bias_position']
    bias_rotation = graph_data.env_data['bias_rotation']
    env_bound = graph_data.env_data['env_bound']
    goal_category = graph_data.goal_cm_info['goal_category']
    
    vis_map = cv2.imread(f'{data_dir}/floor_map/val/{env_name}/floor_map_lv{level}_{goal_category}.png')
    vis_map = cv2.cvtColor(vis_map, cv2.COLOR_BGR2RGB)
    
    vis_map_info = {
        'grid_resolution': np.shape(vis_map),
        'bias_position': bias_position,
        'bias_rotation': bias_rotation,
        'env_bound': env_bound,
    }
    
    return vis_map, vis_map_info

vis_map, vis_map_info = load_floor_map(graph_data, args.data_dir)

NameError: name 'graph_data' is not defined

In [5]:
obj_value = np.squeeze(pred_dist.cpu().numpy(), axis=1)
vis_map = vis_topdown_graph_map(vis_map, graph_data, obj_value, vis_map_info, curr_node_id=graph_data.visited_node_ids[-1])

value_type = 'Value Estimation'
save_name = args.save_dir +'/' + data_name.split('/')[-1]
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)
goal_name = graph_data.goal_cm_info['goal_category']
env_name = data_name.split('/')[-1].split('_')[0]

save_graph_on_topdown_map(vis_map, value_type, save_name, goal_name, env_name, save=False)

NameError: name 'pred_dist' is not defined

In [6]:
def draw_graph_comparison(graph_data, node_goal_dists, info_features, pred_dist, acc, vis_map, vis_map_info, save_name, save=False):
    ## gt_map
    gt_map = np.copy(vis_map)
    gt_value = np.squeeze(node_goal_dists.cpu().numpy(), axis=1)
    gt_map = vis_topdown_graph_map(gt_map, graph_data, gt_value, vis_map_info, curr_node_id=graph_data.visited_node_ids[-1])


    ## cm_map
    cm_map = np.copy(vis_map)
    cm_value = np.max(info_features[:,4:].cpu().numpy(), axis=1)
    cm_map = vis_topdown_graph_map(cm_map, graph_data, cm_value, vis_map_info, curr_node_id=graph_data.visited_node_ids[-1])

    ## value_map
    value_map = np.copy(vis_map)
    obj_value = np.squeeze(pred_dist.cpu().numpy(), axis=1)
    value_map = vis_topdown_graph_map(value_map, graph_data, obj_value, vis_map_info, curr_node_id=graph_data.visited_node_ids[-1])


    save_comparison_on_topdown_map(gt_map, cm_map, value_map, save_name, goal_name, env_name, acc, save=save)

In [7]:
vis_map, vis_map_info = load_floor_map(graph_data, args.data_dir)
draw_graph_comparison(graph_data, node_goal_dists, info_features, pred_dist, acc, vis_map, vis_map_info, save=True)

NameError: name 'graph_data' is not defined

In [8]:
for i in tqdm(range(len(val_list))):
    data_name = val_list[i]
    graph_data, data, features, info_features, goal_features, adj_mtx, node_goal_dists, goal_idx, pred_dist = pred_value(data_name)
    acc = eval_result(data, pred_dist, node_goal_dists)
    save_name = args.save_dir +'/' + data_name.split('/')[-1]
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    vis_map, vis_map_info = load_floor_map(graph_data, args.data_dir)
    draw_graph_comparison(graph_data, node_goal_dists, info_features, pred_dist, acc, vis_map, vis_map_info, save_name, save=True)

NameError: name 'val_list' is not defined

In [ ]:
2*2*np.sin(np.deg2rad(15))